# Capstone Project - The Battle of the Neighborhoods (Week 2)

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
1. Number of existing restaurants in the neighborhood (any type of restaurant)
1. Number of and distance to Chinese restaurants in the neighborhood, if any
distance of neighborhood from city center.

Use regularly spaced grid of locations, centered around city center, to define our neighborhoods.
Following data sources will be needed to extract/generate the required information:
centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using LocationIQ API reverse geocoding
number of restaurants and their type and location in every neighborhood will be obtained using Foursquare API;
coordinate of The Bronx center will be obtained using LocationIQ API geocoding of a well known location (Queens Center)

In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
import requests
import json

def get_coordinates(key, q, verbose=False):
    try:
        url = 'https://us1.locationiq.com/v1/search.php?key={}&q={}&format=json'.format(key, q)
        response = requests.get(url).json()
        if verbose:
            print('API JSON result =>', response)
        results = json.dumps(response)
        boundingbox = response[0]['boundingbox'] # get geographical coordinates
        lat = response[0]['lat']
        lon = response[0]['lon']
        return [lat, lon]
    except:
        return [None, None]

In [3]:
address = 'Queens Center, Queens, NY, US'
ny_center = get_coordinates(key, q)
print('Coordinate of {}: {}'.format(address, ny_center))

Coordinate of Queens Center, Queens, NY, US: ['40.73459645', '-73.8700115945247']


Now let's create a grid of area candidates, equaly spaced, centered around city center and within about 25km from Queens Center. Our neighborhoods will be defined as circular areas with a radius of 1km, so our neighborhood centers will be 2km apart.
To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in meters).

In [4]:
!pip install shapely
import shapely.geometry

!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('NY center longitude = {}, latitude = {}'.format(ny_center[1], ny_center[0]))
x, y = lonlat_to_xy(ny_center[1], ny_center[0])
print('NY center UTM X = {}, Y = {}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('NY center longitude = {}, latitude = {}'.format(lo, la))

     |████████████████████████████████| 1.8MB 8.3MB/s eta 0:00:01
     |████████████████████████████████| 10.4MB 6.8MB/s eta 0:00:01    |██████████▌                     | 3.4MB 6.8MB/s eta 0:00:02
Coordinate transformation check
-------------------------------
NY center longitude = -73.8700115945247, latitude = 40.73459645
NY center UTM X = -5818085.746727511, Y = 9852160.999956807
NY center longitude = -73.87001159452427, latitude = 40.73459644999888


In [5]:
ny_center_x, ny_center_y = lonlat_to_xy(ny_center[1], ny_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = ny_center_x - 25000
x_step = 2000
y_min = ny_center_y - 25000 - (int(25/k)*k*2000 - 50000)/2
y_step = 2000 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 1000 if i%2==0 else 0
    for j in range(0, 25):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(ny_center_x, ny_center_y, x, y)
        if (distance_from_center <= 25001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

506 candidate neighborhood centers generated.


In [6]:
!pip install folium

import folium

     |████████████████████████████████| 92kB 6.8MB/s eta 0:00:011


In [7]:
map_ny = folium.Map(location = ny_center, zoom_start = 13)
folium.Marker(ny_center, popup = 'Queens Center').add_to(map_ny)
for lat, lon in zip(latitudes, longitudes):
    folium.Circle([lat, lon], radius=1000, color = 'red', fill = False).add_to(map_ny)
map_ny

In [8]:
# The code was removed by Watson Studio for sharing.

In [9]:
def get_address(key, lat_x, lon_x, verbose=False):
    try:
        url = 'https://us1.locationiq.com/v1/reverse.php?key={}&lat={}&lon={}&format=json'.format(key, lat_x, lon_x)
        response = requests.request('GET', url=url)
        if verbose:
            print('API JSON result =>', response)
        results = response.json()
        address = results['display_name']
        return address
    except:
        return None

addr = get_address(key, ny_center[0], ny_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(ny_center[0], ny_center[1], addr))

Reverse geocoding check
-----------------------
Address of [40.73459645, -73.8700115945247] is: 90-15, Queens Boulevard, Elmhurst, New York, Queens County, New York, 11373, USA


In [10]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', US', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

In [11]:
addresses[1:50]

['69, Catherine Avenue, Franklin Square, Nassau County, New York, 11010A',
 '26, Jefferson Street, North End, Garden City, Nassau County, New York, 11530A',
 'NO ADDRESS',
 'NO ADDRESS',
 '58, Laurel Drive, North New Hyde Park, Nassau County, New York, 11040A',
 '474, Links Drive South, North Hills, Nassau County, New York, 11576A',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 '31, Tunnel Street, North End, Floral Park, Nassau County, New York, 11001A',
 '905, North 3rd Street, Hillcrest, North New Hyde Park, Nassau County, New York, 11040A',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'Greentree Foundation, Community Drive, Lake Success, Nassau County, New York, 11030A',
 '17, Birch Lane, South Valley Stream, South Valley Stream, Nassau County, New York, 11581A',
 'NO ADDRESS',
 'NO ADDRESS',
 'NO ADDRESS',
 'Foster Meadow Lane, Elmont, Nassau County, New York, 11003A',
 '100, Geranium Avenue, Floral Park, Nassau County, New York, 11001A',
 'NO ADDRESS',
 'NO ADDRESS

In [12]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"Stop & Shop, Franklin Square, Nassau County, N...",40.694604,-73.683814,-5.824086e+06,9.827912e+06,24979.991994
1,"69, Catherine Avenue, Franklin Square, Nassau ...",40.706381,-73.683271,-5.822086e+06,9.827912e+06,24576.411455
2,"26, Jefferson Street, North End, Garden City, ...",40.718160,-73.682729,-5.820086e+06,9.827912e+06,24331.050121
3,NO ADDRESS,40.729942,-73.682185,-5.818086e+06,9.827912e+06,24248.711306
4,NO ADDRESS,40.741727,-73.681642,-5.816086e+06,9.827912e+06,24331.050121
5,"58, Laurel Drive, North New Hyde Park, Nassau ...",40.753515,-73.681098,-5.814086e+06,9.827912e+06,24576.411455
6,"474, Links Drive South, North Hills, Nassau Co...",40.765305,-73.680554,-5.812086e+06,9.827912e+06,24979.991994
7,NO ADDRESS,40.677300,-73.698017,-5.827086e+06,9.829644e+06,24248.711306
8,NO ADDRESS,40.689073,-73.697480,-5.825086e+06,9.829644e+06,23579.652245
9,NO ADDRESS,40.700848,-73.696944,-5.823086e+06,9.829644e+06,23065.125189


In [13]:
df_locations.to_pickle('./locations.pkl')

Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.
We're interested in venues in 'food' category, so we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Chinese restaurant' category, as we need info on Chinese restaurants in the neighborhood.

In [14]:
# @hiddencell
client_id = 'I1QQKNVV4PQXBZXEU2CHBLZIMLD4LBTBODR12CEGF23ASPMY'
client_secret = 'FORSGYRRFIEKBNRRPJ0YT4BIQ14BOEZ2222U52P0KGYKFWET'

In [15]:
# Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

chinese_restaurant_categories = ['4bf58dd8d48988d145941735','52af3a5e3cf9994f4e043bea','52af3a723cf9994f4e043bec',
                                 '52af3a7c3cf9994f4e043bed','58daa1558bbb0b01f18ec1d3','52af3a673cf9994f4e043beb',
                                 '52af3a903cf9994f4e043bee','4bf58dd8d48988d1f5931735','52af3a9f3cf9994f4e043bef',
                                 '52af3aaa3cf9994f4e043bf0','52af3ab53cf9994f4e043bf1','52af3abe3cf9994f4e043bf2',
                                 '52af3ac83cf9994f4e043bf3','52af3ad23cf9994f4e043bf4','52af3add3cf9994f4e043bf5',
                                 '52af3af23cf9994f4e043bf7','52af3ae63cf9994f4e043bf6','52af3afc3cf9994f4e043bf8',
                                 '52af3b053cf9994f4e043bf9','52af3b213cf9994f4e043bfa','52af3b293cf9994f4e043bfb',
                                 '52af3b343cf9994f4e043bfc','52af3b3b3cf9994f4e043bfd','52af3b463cf9994f4e043bfe',
                                 '52af3b633cf9994f4e043c01','52af3b513cf9994f4e043bff','52af3b593cf9994f4e043c00',
                                 '52af3b6e3cf9994f4e043c02','52af3b773cf9994f4e043c03','52af3b813cf9994f4e043c04',
                                 '52af3b893cf9994f4e043c05','52af3b913cf9994f4e043c06','52af3b9a3cf9994f4e043c07',
                                 '52af3ba23cf9994f4e043c08']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Chinese', '')
    address = address.replace(', China', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20200314'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [16]:
# Let's now go over our neighborhood locations and get nearby restaurants; 
#we'll also maintain a dictionary of all found restaurants and all found chinese restaurants

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    chinese_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage 
        #so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, client_id, client_secret, radius=350, 
                                          limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_chinese = is_restaurant(venue_categories, specific_filter = chinese_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], 
                              venue_address, venue_distance, is_chinese, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_chinese:
                    chinese_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, chinese_restaurants, location_restaurants

# Try to load from local file system, in case we did this before
restaurants = {}
chinese_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('chinese_restaurants_350.pkl', 'rb') as f:
        chinese_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed, use the Foursquare API to get the data
if not loaded:
    restaurants, chinese_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('chinese_restaurants_350.pkl', 'wb') as f:
        pickle.dump(chinese_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

In [17]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Chinese restaurants:', len(chinese_restaurants))
print('Percentage of Chinese restaurants: {:.2f}%'.format(len(chinese_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 2181
Total number of Chinese restaurants: 337
Percentage of Chinese restaurants: 15.45%
Average number of restaurants in neighborhood: 3.2549407114624507


In [18]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('53c3cdf8498efcce8ee0189e', 'Higuey Cafe Restaurant', 40.708586886710904, -73.68484714284935, '12-04 Hempstead Toke (Frannklin Square), Franklin Square, NY 11010, United States', 279, False, -5821718.754869538, 9828128.765375335)
('4e4e3a57bd4101d0d7a55bb4', 'Mama Food', 40.70865308379805, -73.6849965442205, '1212 Hempstead Tpke, Franklin Square, NY 11010, United States', 291, False, -5821708.20348589, 9828148.439533973)
('5913ac1f4420d82816cc34bd', 'Olde Trading Post', 40.73299537565128, -73.6829094883608, '1218 Jericho Tpke (Millers Lane), New Hyde Park, NY 11040, United States', 345, False, -5817571.498100383, 9828023.805070203)
('50981c15e4b07ae1b90b21f2', 'Kitchen\uf44d', 40.765807350192034, -73.67726105847183, 'Roslyn, NY 11576, United States', 283, False, -5811985.79065345, 9827491.14605183)
('4c813eb4d4e237048bee5b88', 'Great Wall Chinese Restaurant', 40.687024129721834, -73.69949895494389, '1629 Dutch Broadway, Elmont, NY 11003,

In [19]:
print('List of Chinese restaurants')
print('---------------------------')
for r in list(chinese_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(chinese_restaurants))

List of Chinese restaurants
---------------------------
('4c813eb4d4e237048bee5b88', 'Great Wall Chinese Restaurant', 40.687024129721834, -73.69949895494389, '1629 Dutch Broadway, Elmont, NY 11003, United States', 284, True, -5825442.351875132, 9829892.991673566)
('4c4b33526e2095215d11fceb', 'Hing Long', 40.681683162595135, -73.71172585400232, '40 Central Ct, Valley Stream, NY 11580, United States', 212, True, -5826403.06295807, 9831441.073906226)
('4bdb177c63c5c9b6fd4a2668', 'East Shore Wok', 40.78864161810552, -73.70704710327456, '73 E Shore Rd, Manhasset, NY 11030, United States', 122, True, -5808251.8420860255, 9831460.526578847)
('4bd08bd8462cb713a392d807', 'Szechuan Delight', 40.68958888098799, -73.72213668665793, '240 Linden Blvd, Elmont, NY 11003, United States', 181, True, -5825107.599983688, 9832831.638197267)
('4e4dfb2abd4101d0d79f8d98', "Kam's Garden Chinese Restaurant", 40.72489706247307, -73.7226826275663, '245-07 Jericho Tpke, Bellerose, NY 11426, United States', 23, Tru

In [20]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 130):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: 
Restaurants around location 102: 
Restaurants around location 103: 
Restaurants around location 104: 
Restaurants around location 105: 
Restaurants around location 106: 
Restaurants around location 107: 
Restaurants around location 108: 
Restaurants around location 109: 
Restaurants around location 110: Caviar House & Prunier, The Paris Cafe, Bobby Van's Steakhouse, Wok & Roll, Dromos Mediterranean, Dromos Mediterranean, Mango Taco
Restaurants around location 111: 
Restaurants around location 112: Fairfield Inn Breakfast/Mediterranean Grill, China Inn Express, Kennedy deli, NAAN & GRILL
Restaurants around location 113: The Door
Restaurants around location 114: Kam Hung, Cheung Hing Kitchen, Nations Fish Emporium
Restaurants around location 115: 
Restaurants around location 116: Halal Chinese Kitchen, Sunshine Grill and Restaurant, Foo On Restaurant
Restaurants around location 117: 
Restaurants aro

In [21]:
map_ny = folium.Map(location = ny_center, zoom_start=13)
folium.Marker(ny_center, popup = 'Queens Center').add_to(map_ny)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_chinese = res[6]
    color = 'red' if is_chinese else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_ny)
map_ny